PROBLEM B4
Build and train a classifier for the BBC-text dataset.

This is a multiclass classification problem.

Do not use lambda layers in your model.

The dataset used in this problem is originally published in: http://mlg.ucd.ie/datasets/bbc.html.

Desired accuracy and validation_accuracy > 91%

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import pandas as pd
import csv
import urllib
import numpy as np

In [2]:
bbc = ('https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/bbc-text.csv')
urllib.request.urlretrieve(bbc, 'bbc-text.csv')

('bbc-text.csv', <http.client.HTTPMessage at 0x27d6c1b68f0>)

In [3]:
# DO NOT CHANGE THIS CODE
# Make sure you used all of these parameters or you can not pass this test
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [4]:
# YOUR CODE HERE
# Using "shuffle=False"
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between",
                "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd",
                "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it",
                "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out",
                "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves",
                "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", 
                "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", 
                "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

sentences = []
labels =[]

with open('bbc-text.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)


In [5]:
train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]


In [6]:
# Fit your tokenizer with training data
# You can also use Tokenizer to encode your label.
# YOUR CODE HERE
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [7]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

In [8]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

train_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [9]:
model = tf.keras.Sequential([
    # YOUR CODE HERE.
    # YOUR CODE HERE. DO not change the last layer or test may fail
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.fit(train_padded, train_label_seq,
          epochs=50,
          validation_data = (validation_padded, validation_label_seq))

Epoch 1/50
56/56 [==============================] - 2s 12ms/step - loss: 1.7650 - accuracy: 0.2281 - val_loss: 1.7320 - val_accuracy: 0.2270
Epoch 2/50
56/56 [==============================] - 0s 7ms/step - loss: 1.6864 - accuracy: 0.2303 - val_loss: 1.6415 - val_accuracy: 0.2292
Epoch 3/50
56/56 [==============================] - 0s 7ms/step - loss: 1.5753 - accuracy: 0.2702 - val_loss: 1.5215 - val_accuracy: 0.3685
Epoch 4/50
56/56 [==============================] - 1s 10ms/step - loss: 1.4269 - accuracy: 0.4888 - val_loss: 1.3616 - val_accuracy: 0.5461
Epoch 5/50
56/56 [==============================] - 0s 7ms/step - loss: 1.2396 - accuracy: 0.6135 - val_loss: 1.1726 - val_accuracy: 0.6247
Epoch 6/50
56/56 [==============================] - 0s 7ms/step - loss: 1.0432 - accuracy: 0.7348 - val_loss: 0.9957 - val_accuracy: 0.7910
Epoch 7/50
56/56 [==============================] - 0s 8ms/step - loss: 0.8653 - accuracy: 0.8489 - val_loss: 0.8392 - val_accuracy: 0.8360
Epoch 8/50
56/56 [

In [11]:
model.save("model_B4.h5")